# Distance calculation between WAMEX coordonates and Minedex nickle deposits

In [6]:
import pandas as pd
import numpy as np
import geopy.distance
import fiona
from tqdm import tqdm

### WA nickle deposits locations (from Minedex)

####  Shape file gets converted to CSV

In [15]:
!ls ../../data/SHP/

MINEDEX - Ni-Cu-Co deposits.cpg
MINEDEX - Ni-Cu-Co deposits.dbf
MINEDEX - Ni-Cu-Co deposits.prj
MINEDEX - Ni-Cu-Co deposits.shp
MINEDEX - Ni-Cu-Co deposits.shx


In [16]:
SHAPE_PATH = '../../data/SHP/MINEDEX - Ni-Cu-Co deposits.shp'

In [18]:
shape = fiona.open(SHAPE_PATH)
# shape.next()['properties']

In [19]:
ordered_dicts = []

for item in shape.items():
    ordered_dicts.append(item[1])

In [20]:
df = pd.DataFrame([ordered_dicts[i]['properties'] for i, _ in enumerate(ordered_dicts)])
df.head()

,SITE_CODE,SITE_TITLE,SHORT_NAME,SITE_COMMO,SITE_TYPE_,SITE_SUB_T,SITE_STAGE,TARGET_COM,COMMODITY,PROJ_CODE,PROJ_TITLE,CONFIDENTI,POINT_CONF,LATITUDE,LONGITUDE,WEB_LINK,EXTRACT_DA,MIN_TYPE
0,None,Andover VC-07,Andover VC-07,"Ni, Cu, Co",Occurrence,Drillhole,Exploration,NICKEL,STEEL ALLOY METAL,None,None,Public,Public,NaN,NaN,None,None,Orthomagmatic - mafic intrusion
1,None,Andover VC-23,Andover VC-23,"Ni, Cu, Co",Occurrence,Drillhole,Exploration,NICKEL,STEEL ALLOY METAL,None,None,None,None,NaN,NaN,None,None,Orthomagmatic - mafic intrusion
2,S0000192,Flying Fox Old - OTZ / Outokumpu,Flying Fox Old,"Ni, Cu",Mine,Decline,Shut,NICKEL,STEEL ALLOY METAL,J00757,Forrestania Nickel / Western Areas,Public,Public,-32.416982,119.689058,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,None
3,S0000193,West Quest Nickel,West Quest Ni,"Ni, Cu",Prospect,Drillhole,Undeveloped,NICKEL,STEEL ALLOY METAL,J00757,Forrestania Nickel / Western Areas,Public,Public,-32.263890,119.770800,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,None
4,S0001174,Halleys,Halleys,"Ni, Co, Mg",Mine,Openpit,Operating,NICKEL,STEEL ALLOY METAL,J02145,Ravensthorpe - First Quantum,Public,Public,-33.633382,120.374303,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,None


In [21]:
df['SITE_TYPE_'].value_counts()

Mine          403
Prospect      349
Deposit       221
Occurrence    124
Name: SITE_TYPE_, dtype: int64

In [23]:


#save df
df.to_csv('../../data/minedex/Mindex_Ni_Cu_Co_deposits.csv')

In [24]:
#re-open
df_minedex = pd.read_csv('../../data/minedex/Mindex_Ni_Cu_Co_deposits.csv')

#### Filter the csv

We will further filter prospects and occurences out as suggested by Trevor. 

In [25]:
df_minedex = df_minedex.loc[(df_minedex['SITE_TYPE_'] == 'Mine') | (df_minedex['SITE_TYPE_'] == 'Deposit')]

In [26]:
df_minedex.shape

(624, 19)

In [27]:
df_minedex.head()

,Unnamed: 0,SITE_CODE,SITE_TITLE,SHORT_NAME,SITE_COMMO,SITE_TYPE_,SITE_SUB_T,SITE_STAGE,TARGET_COM,COMMODITY,PROJ_CODE,PROJ_TITLE,CONFIDENTI,POINT_CONF,LATITUDE,LONGITUDE,WEB_LINK,EXTRACT_DA,MIN_TYPE
2,2,S0000192,Flying Fox Old - OTZ / Outokumpu,Flying Fox Old,"Ni, Cu",Mine,Decline,Shut,NICKEL,STEEL ALLOY METAL,J00757,Forrestania Nickel / Western Areas,Public,Public,-32.416982,119.689058,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,NaN
4,4,S0001174,Halleys,Halleys,"Ni, Co, Mg",Mine,Openpit,Operating,NICKEL,STEEL ALLOY METAL,J02145,Ravensthorpe - First Quantum,Public,Public,-33.633382,120.374303,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,NaN
5,5,S0001178,Black Swan - Gosling,Black Swan,Ni,Mine,Openpit,Care and Maintenance,NICKEL,STEEL ALLOY METAL,J00452,Silver Swan - Black Swan,Public,Public,-30.393502,121.648875,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,NaN
6,6,S0001188,Avalon Group,Bulong,"Ni, Co",Mine,Openpit,Care and Maintenance,NICKEL,STEEL ALLOY METAL,J00461,Avalon,Public,Public,-30.736222,121.802770,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,NaN
7,7,S0001197,Byro East,Byro East,"Ni, Cu, Au, Pt, Pd, Cr",Deposit,Unspecified,Undeveloped,NICKEL,STEEL ALLOY METAL,J00470,Byro Nickel,Public,Public,-26.037849,116.387001,https://minedex.dmirs.wa.gov.au/Web/common/jum...,2021-07-28,NaN


In [28]:
#Count by site type
df_minedex['SITE_TYPE_'].value_counts()

Mine       403
Deposit    221
Name: SITE_TYPE_, dtype: int64

In [30]:
# df_minedex['SITE_COMMO'].value_counts()

#### Get coodonates

In [41]:
data = {'minedex_lat': df_minedex['LATITUDE'], 'minedex_lon': df_minedex['LONGITUDE'], 'minedex_site': df_minedex['SITE_CODE']}

In [42]:
mindex_coords_df = pd.DataFrame(data)

In [43]:
mindex_coords_df.head()

,minedex_lat,minedex_lon,minedex_site
2,-32.416982,119.689058,S0000192
4,-33.633382,120.374303,S0001174
5,-30.393502,121.648875,S0001178
6,-30.736222,121.802770,S0001188
7,-26.037849,116.387001,S0001197


In [45]:
mindex_coords_df.shape

(624, 3)

In [46]:
#there are as many different mine sites locations as rows
mindex_coords_df['minedex_site'].nunique()

624

In [47]:
mindex_coords = list(zip(mindex_coords_df['minedex_lat'], mindex_coords_df['minedex_lon']))
mindex_coords[1:5]

[(-33.633382, 120.374303),
 (-30.393502, 121.648875),
 (-30.736222, 121.80277),
 (-26.037849, 116.387001)]

In [48]:
mindex_coords_df['coords'] = mindex_coords

In [49]:
mindex_coords_df.head()

,minedex_lat,minedex_lon,minedex_site,coords
2,-32.416982,119.689058,S0000192,"(-32.416982, 119.689058)"
4,-33.633382,120.374303,S0001174,"(-33.633382, 120.374303)"
5,-30.393502,121.648875,S0001178,"(-30.393502, 121.648875)"
6,-30.736222,121.802770,S0001188,"(-30.736222, 121.80277)"
7,-26.037849,116.387001,S0001197,"(-26.037849, 116.387001)"


## WAMEX assays

In [2]:
df_wamex = pd.read_csv('WAMEX_with_Co_Ni_MINDEX_available_minerals.csv')
df_wamex.shape

(179193, 13)

In [3]:
df_wamex.head()

,Unnamed: 0,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag,lat,long
0,33,56.0,420.0,9.0,14.0,0.010,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146
1,34,31.0,195.0,6.0,10.0,0.007,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146
2,35,90.0,150.0,15.0,2.0,0.004,0.5,NaN,NaN,NaN,NaN,-17.234642,128.032146
3,36,37.0,135.0,11.0,4.0,0.005,1.0,NaN,NaN,NaN,NaN,-17.216592,128.047184
4,37,58.0,120.0,14.0,5.0,0.004,0.5,NaN,NaN,NaN,NaN,-17.216028,128.048165


In [56]:
wamex_coords = list(zip(df_wamex['lat'], df_wamex['long']))
wamex_coords[1:5]

[(-17.234642, 128.032146),
 (-17.234642, 128.032146),
 (-17.216592, 128.047184),
 (-17.216028, 128.048165)]

In [57]:
len(wamex_coords)

179193

In [58]:
wamex_coords_nodupl = tuple(set(wamex_coords))
len(wamex_coords_nodupl)

133965

## Calculate Distance

In [59]:
new = pd.DataFrame(columns=['minedex_coords', 'minedex_site', 'wamex_lat', 'wamex_lon', 'distance',])

In [60]:
#simple distance function try
def calc_distance(coords_a, coords_b):
       dist = geopy.distance.distance(coords_a, coords_b)
       return(dist)

In [61]:
calc_distance((-17.234642, 128.032146), (-26.78737, 118.574407))

Distance(1438.3974155883463)

In [62]:
wamex_coords_nodupl[1]

(-25.585249, 116.825226)

In [63]:
for i in tqdm(range(0, len(wamex_coords_nodupl))):
    for j in range(0, len(mindex_coords_df)):
            comp = mindex_coords_df.iloc[j]['coords']
            dist = calc_distance(wamex_coords_nodupl[i], comp)
            if dist < 15:
                new_row = {'minedex_coords': comp, 'minedex_site': mindex_coords_df.iloc[j]['minedex_site'],'wamex_lat':wamex_coords_nodupl[i][0],'wamex_lon': wamex_coords_nodupl[i][1], 'distance':dist}
                new = new.append(new_row, ignore_index=True)

100%|████████████████████████████████████████████████████████████████████████| 133965/133965 [8:55:05<00:00,  4.17it/s]


In [64]:
len(new)

111302

In [65]:
new.to_csv('Distance_MinedexNi_WAMEX.csv')

In [67]:
new.head()

,minedex_coords,minedex_site,wamex_lat,wamex_lon,distance
0,"(-31.591497, 123.441046)",S0236838,-31.500331,123.406984,10.613232589537946 km
1,"(-21.656719, 120.386398)",S0016658,-21.672418,120.369385,2.474348626310701 km
2,"(-29.74929, 120.559601)",S0021377,-29.835919,120.619067,11.19239782986901 km
3,"(-20.92964, 120.207901)",S0023507,-20.891481,120.130476,9.095442991298277 km
4,"(-26.885526, 122.214074)",S0020422,-26.824898,122.186901,7.240139569880715 km


The result includes duplicates of WAMEX assays because some assays are close to several MINEDEX mine sites. 

### Merge with actual assays

In [4]:
distances = pd.read_csv('Distance_MinedexNi_WAMEX.csv')

In [5]:
len(distances)

111302

In [9]:
distances.head()

,Unnamed: 0,minedex_coords,minedex_site,wamex_lat,wamex_lon,distance
0,0,"(-31.591497, 123.441046)",S0236838,-31.500331,123.406984,10.613232589537946 km
1,1,"(-21.656719, 120.386398)",S0016658,-21.672418,120.369385,2.474348626310701 km
2,2,"(-29.74929, 120.559601)",S0021377,-29.835919,120.619067,11.19239782986901 km
3,3,"(-20.92964, 120.207901)",S0023507,-20.891481,120.130476,9.095442991298277 km
4,4,"(-26.885526, 122.214074)",S0020422,-26.824898,122.186901,7.240139569880715 km


In [10]:
df_wamex.head()

,Unnamed: 0,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag,lat,long
0,33,56.0,420.0,9.0,14.0,0.010,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146
1,34,31.0,195.0,6.0,10.0,0.007,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146
2,35,90.0,150.0,15.0,2.0,0.004,0.5,NaN,NaN,NaN,NaN,-17.234642,128.032146
3,36,37.0,135.0,11.0,4.0,0.005,1.0,NaN,NaN,NaN,NaN,-17.216592,128.047184
4,37,58.0,120.0,14.0,5.0,0.004,0.5,NaN,NaN,NaN,NaN,-17.216028,128.048165


In [12]:
Merged_Ni_lessthan15km = pd.merge(df_wamex, distances, how='inner', left_on=['lat', 'long'], right_on=['wamex_lat', 'wamex_lon'])

In [13]:
Merged_Ni_lessthan15km.head()

,Unnamed: 0_x,Ni,Cu,Co,Pd,Au,Pt,Rh,Fe,S,Ag,lat,long,Unnamed: 0_y,minedex_coords,minedex_site,wamex_lat,wamex_lon,distance
0,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46661,"(-17.34893, 128.022095)",S0001442,-17.234642,128.032146,12.693593881734987 km
1,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46662,"(-17.31078, 128.023499)",S0002890,-17.234642,128.032146,8.476376684524638 km
2,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46663,"(-17.3526, 128.027695)",S0019720,-17.234642,128.032146,13.063277176872425 km
3,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46664,"(-17.3526, 128.027695)",S0019821,-17.234642,128.032146,13.063277176872425 km
4,33,56.0,420.0,9.0,14.0,0.01,1.0,NaN,NaN,NaN,NaN,-17.234642,128.032146,46665,"(-17.338411, 128.017007)",S0231765,-17.234642,128.032146,11.59661465949521 km


In [16]:
Merged_Ni_lessthan15km.to_csv('Merged_Ni_lessthan15km.csv')